<a href="https://colab.research.google.com/github/m-zaniolo/CEE690-ESAA/blob/main/Lab_5_Direct_Policy_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5 - Direct policy search

Load data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 4)


url = 'https://raw.githubusercontent.com/m-zaniolo/CEE690-ESAA/main/data/'
inflow = np.loadtxt(url + 'inflow.txt', delimiter='\t') #m3/s

# generate date array
# Generate the full date range from January 1, 2001, to December 31, 2015
date_range = pd.date_range(start='2001-01-01', end='2015-12-31', freq='D')

# Filter out all February 29 instances
date_range = date_range[~((date_range.month == 2) & (date_range.day == 29))]



---



Let's code a standard operating policy where the release decision u(t) is a function of the storage s(t)

In [ ]:
sim_step = 60*60*24
S = 1e10 #reservoir max capacity
D = 200*sim_step    # water demand, m3/day.

#let's work in m3/day just so parameters are easier to type.

#operating policy #1
theta = [0.004, -3e7, 0.007]


s_values = np.arange(0, S, 10e3) #just over max storage
L1 = theta[0]*s_values
L2 = D*np.ones((len(s_values),))
L3 = theta[1] + s_values*theta[2]

fig, ax = plt.subplots()
plt.plot(s_values, L1, color = 'r')
plt.plot(s_values, L2, color = 'y')
plt.plot(s_values, L3, color = 'g')
plt.ylim(0, np.max(L3))
plt.ylabel('release [m3/day]')
plt.xlabel('storage [m3]')


release decision is determined as:

In [ ]:
u_t = np.maximum( np.minimum(L1, L2), L3 ) # element-wise max/minimum
max_release = 400 * sim_step

plt.plot(s_values, u_t)
plt.plot(s_values, s_values*(max_release/S), '--')

r_tp = np.maximum( np.minimum(u_t, s_values*(max_release/S)), 0)

plt.plot(s_values, r_tp)
plt.legend(['u_t', 'physical constraints', 'r_t+1'])

#how does this policy perform?

1) Define parameters for reservoir simulation

In [ ]:
H = len(inflow) #length of the simulation horizon
evap = np.loadtxt(url + 'netevap_Gibe1.txt', delimiter=' ') # load cyclostationary evaporation

s0 = S*0.7 # for example, let's inizialize storage value at 70% of the max capacity
stor_to_surface = 0.0142
# stor_to_level = 0.0521*storage^0.3589

Ny = H/365



2) implement reservoir simulation function:

In [ ]:
def simulate(theta):

  # time convention
  inflow_ = np.concatenate(([-999], inflow)) * sim_step

  # initialize storage vector
  s = np.zeros(len(inflow_))
  l = np.zeros(len(inflow_)) ### new
  r = np.zeros(len(inflow_)) ### new
  u = np.zeros(len(inflow_)) ### new

  s[0] = S*0.7
  l[0] = 0.0521*(s[0]**0.3589)   ### calculate level


  # mass balance
  for t in range(H):
    evaporation_t = evap[t%365]/1000*s[t]*stor_to_surface

    #### new - policy as a piecewise linear function. Substitute s_values with s[t]
    L1 = theta[0]*s[t]
    L2 = D
    L3 = theta[1] + s[t]*theta[2]

    u[t] = max( min(L1, L2), L3 )
    r[t+1] = max( min( u[t], s[t]*(max_release/S)),  0)

    s[t+1] = s[t] + inflow_[t+1] - r[t+1] - evaporation_t # we corrected for sim_step above

    # check minimum storage constraint
    s[t+1] = max(0, s[t+1])
    l[t+1] = 0.0521*(s[t+1]**0.3589)   ### calculate level
    s[t+1] = min(s[t+1], S) #check max storage constraint

  #indicator 1: flood days
  l_threshold = 0.0521*(S**0.3589) #obtain maximum level threshold from max storage
  i1 = np.sum(l > l_threshold)/Ny

  #indicator 2: water demand deficit
  i2 = sum ( (np.maximum(D - r[1:] , 0)/sim_step) **2 ) / Ny

  return i1, i2, s[:-1], l[:-1], r[1:], u[:-1]



In [ ]:
# call this function as:
i1, i2, s, l, r, u = simulate(theta)
plt.subplot(3,1,1)
plt.plot(date_range, l)
l_threshold = 0.0521*(S**0.3589)
plt.plot(date_range, l_threshold*np.ones(H), '--')
plt.legend(['level', 'flood threshold'])
plt.ylim([0, 210])


plt.subplot(3,1,2)
plt.plot(date_range, inflow*sim_step)
plt.plot(date_range, r)
plt.plot(date_range, u)
plt.plot(date_range, D*np.ones(H), '--')
plt.legend(['inflow', 'r[t+1]', 'u[t]', 'demand'])




print(i1, i2)

In [ ]:
plt.scatter(s, u/sim_step)
plt.xlim([0,S])

###Let's use MOEAs to optimize policy parameters

In [ ]:
!pip install platypus-opt

Rewrite the simulation function with just indicators as outputs

In [ ]:
def simulate_MOEA(theta):

  # time convention
  inflow_ = np.concatenate(([-999], inflow)) * sim_step

  # initialize storage vector
  s = np.zeros(len(inflow_))
  l = np.zeros(len(inflow_)) ### new
  r = np.zeros(len(inflow_)) ### new

  s[0] = S*0.7
  l[0] = 0.0521*(s[0]**0.3589)   ### calculate level

  # mass balance
  for t in range(H):
    evaporation_t = evap[t%365]/1000*s[t]*stor_to_surface

    #### new - policy as a piecewise linear function. Substitute s_values with s[t]
    L1 = theta[0]*s[t]
    L2 = D
    L3 = theta[1] + s[t]*theta[2]

    u_t = np.maximum( np.minimum(L1, L2), L3 ) # element-wise max/minimum
    r[t+1] = max( min( u_t, s[t]*(max_release/S)),  0)

    s[t+1] = s[t] + inflow_[t+1] - r[t+1] - evaporation_t # we corrected for sim_step above

    # check minimum storage constraint
    s[t+1] = max(0, s[t+1])
    l[t+1] = 0.0521*(s[t+1]**0.3589)   ### calculate level
    s[t+1] = min(s[t+1], S) #check max storage constraint

  #indicator 1: flood days
  l_threshold = 0.0521*(S**0.3589) #obtain maximum level threshold from max storage
  i1 = np.sum(l > l_threshold)/Ny

  #indicator 2: water demand deficit
  i2 = sum ( (np.maximum(D - r[1:] , 0)/sim_step) **2 ) / Ny

  return i1, i2

In [ ]:
from platypus import NSGAII, Problem, Real

problem = Problem(3, 2) # three decision variables, 2 objectives
lb = [0, -1e10, 0]
ub = [0.1, 0, 0.1]


problem.types[0] = Real(lb[0], ub[0])
problem.types[1] = Real(lb[1], ub[1])
problem.types[2] = Real(lb[2], ub[2])

problem.function = simulate_MOEA
algorithm = NSGAII(problem)

#Optimize the problem using max_NFE function evaluations
max_NFE = 1000 # try changing this!
algorithm.run(max_NFE)

#Convert results to numpy arrays

J = np.array([s.objectives for s in algorithm.result])
theta_sol = np.array([s.variables for s in algorithm.result])


plt.figure(figsize=(8, 3))
plt.scatter(J[:,0],J[:,1])
plt.xlabel('flood days')
plt.ylabel('agricultural deficit')
plt.title('Pareto Front')
plt.show()

Your solution set may contain dominated or semi-dominated solutions.
We can use the function generated last time to select non-dominated solutions only.

In [ ]:
#Identifying the Pareto front from a set of possible points. Assumes minimization

def dominates(a,b): # function that checks if a dominates b
  return (np.all(a <= b) and np.any(a < b))

#The function pareto_sort accepts a matrix of points, returns a matrix of only the nondominated ones. It is not the most efficient method to do this.
#keep is an array of booleans used to index the matrix at the end

def pareto_sort(P):
    N = len(P)
    keep = np.ones(N, dtype=bool) # all True to start

    for i in range(N):
        for j in range(i+1,N):
            if keep[j] and dominates(P[i,:], P[j,:]):
                keep[j] = False

            elif keep[i] and dominates(P[j,:], P[i,:]):
                keep[i] = False

    return P[keep,:] # only keeping the nondominated points

In [ ]:
J_opt = pareto_sort(J)

plt.figure(figsize=(8, 3))
plt.scatter(J_opt[:,0],J_opt[:,1])
plt.xlabel('flood days')
plt.ylabel('agricultural deficit')
plt.title('Pareto Front')
plt.show()

In [ ]:
print(theta_sol[0])

In [ ]:
theta_visualize = theta_sol[0]

i1, i2, s, l, r, u = simulate(theta_visualize)
plt.figure(figsize=(8, 8))

plt.subplot(3,1,1)
plt.plot(r/sim_step)
plt.plot(inflow)
plt.plot(D*np.ones(H)/sim_step, '--')

plt.subplot(3,1,2)
plt.plot(l)
plt.ylim([0,210])
plt.plot(l_threshold*np.ones(H))

plt.subplot(3,1,3)
L1 = theta_visualize[0]*s_values
L2 = D*np.ones((len(s_values),))
L3 = theta_visualize[1] + s_values*theta_visualize[2]

u_t = np.maximum( np.minimum(L1, L2), L3 ) # element-wise max/minimum
r_tp = np.maximum( np.minimum(u_t, s_values*(max_release/S)), 0)

plt.plot(s_values, u_t)
plt.plot(s_values, s_values*(max_release/S))
plt.plot(s_values, r_tp)
plt.ylim([0,1e8])
plt.legend(['u_t', 'physical constraints', 'r_t+1'])


print(i1, i2)

What if we considered more objectives?

In [ ]:
def simulate_MOEA_4obj(theta):

  # time convention
  inflow_ = np.concatenate(([-999], inflow)) * sim_step

  # initialize storage vector
  s = np.zeros(len(inflow_))
  l = np.zeros(len(inflow_)) ### new
  r = np.zeros(len(inflow_)) ### new

  s[0] = S*0.7
  l[0] = 0.0521*(s[0]**0.3589)   ### calculate level

  # mass balance
  for t in range(H):
    evaporation_t = evap[t%365]/1000*s[t]*stor_to_surface

    #### new - policy as a piecewise linear function. Substitute s_values with s[t]
    L1 = theta[0]*s[t]
    L2 = D
    L3 = theta[1] + s[t]*theta[2]

    u_t = np.maximum( np.minimum(L1, L2), L3 ) # element-wise max/minimum
    r[t+1] = max( min( u_t, s[t]*(max_release/S)),  0)

    s[t+1] = s[t] + inflow_[t+1] - r[t+1] - evaporation_t # we corrected for sim_step above

    # check minimum storage constraint
    s[t+1] = max(0, s[t+1])
    l[t+1] = 0.0521*(s[t+1]**0.3589)   ### calculate level
    s[t+1] = min(s[t+1], S) #check max storage constraint

  #indicator 1: flood days
  l_threshold = 0.0521*(S**0.3589) #obtain maximum level threshold from max storage
  i1 = np.sum(l > l_threshold)/Ny

  #indicator 2: water demand deficit
  i2 = sum ( (np.maximum(D - r[1:] , 0)/sim_step) **2 ) / Ny

  #indicator 3: hydropower production


  #indicator 4: minimize annual oscillation in lake level (e.g., for navigation)


  return i1, i2, i3, i4


In [ ]:
#test with policies you generated before
i1, i2, i3, i4 = simulate_MOEA_4obj(theta_sol[0])

## Now use DPS optimal solutions according to 4 objectives

In [ ]:
problem = ???


## display your solutions with a parallel plot

In [ ]:
# Normalize the data to 0-1 range for each dimension.
data_normalized = (J - J.min(axis=0)) / (J.max(axis=0) - J.min(axis=0))

# Number of variables
num_vars = J.shape[1]

# Set up the plot
fig, ax = plt.subplots(figsize=(10, 4))

# Get a colormap (Blues) and normalize the first column for color mapping
norm = plt.Normalize(J[:, 0].min(), J[:, 0].max())
cmap = plt.cm.plasma

# Plot each data point with color based on the value in the first column
for i in range(len(data_normalized)):
    color = cmap(norm(J[i, 0]))  # Get color for each line based on the first column
    ax.plot(range(num_vars), data_normalized[i, :], marker='o', color=color)




# Set the ticks and labels
ax.set_xticks(range(num_vars))
ax.set_xticklabels(['flood days', 'water deficit', 'hydropower', 'level oscillation'])
ax.set_xlim([0, num_vars - 1])

# Add grid lines for better readability
ax.grid(True)

# Set axis labels
ax.set_xlabel('Variables')
ax.set_ylabel('Normalized Value (direction of preference down)')

# Add a title
plt.title('Parallel Coordinates Plot')

plt.show()
